## IMPORT

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 462 kB 41.9 MB/s 
     |████████████████████████████████| 4.2 MB 5.0 MB/s 


In [2]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

In [3]:
# cd /content/drive/Othercomputers/My Laptop/Bangkit/Capstone/Recommender system/Notogo-ML/model_v2.0 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd /content/drive/MyDrive/Bangkit/Capstone/ML/ML-GH/Notogo-ML/"model_v2.1"

/content/drive/MyDrive/Bangkit/Capstone/ML/ML-GH/Notogo-ML/model_v2.1


In [6]:
print(os.getcwd())

/content/drive/MyDrive/Bangkit/Capstone/ML/ML-GH/Notogo-ML/model_v2.1


## Preparing the dataset

In [7]:
import userEmbedding
builder = tfds.builder('Userembedding')
userEmbeddingDs = tfds.load('Userembedding',split='train')

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/userembedding/1.0.0.incompleteZ9FZ8D/userembedding-train.tfrecord*...:   0…

Dataset userembedding downloaded and prepared to /root/tensorflow_datasets/userembedding/1.0.0. Subsequent calls will reuse this data.


In [8]:
import userFeatures
builder = tfds.builder('Userfeatures')
userFeatureDs = tfds.load('Userfeatures',split='train')

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/userfeatures/1.0.0.incompleteXD7Q2Q/userfeatures-train.tfrecord*...:   0%|…

Dataset userfeatures downloaded and prepared to /root/tensorflow_datasets/userfeatures/1.0.0. Subsequent calls will reuse this data.


In [9]:
import wishEmbedding
builder = tfds.builder('Wishembedding')
wishEmbeddingDs = tfds.load('Wishembedding',split='train')

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/wishembedding/1.0.0.incomplete6RCHV2/wishembedding-train.tfrecord*...:   0…

Dataset wishembedding downloaded and prepared to /root/tensorflow_datasets/wishembedding/1.0.0. Subsequent calls will reuse this data.


In [10]:
import wishFeatures
builder = tfds.builder('Wishfeatures')
wishFeaturesDs = tfds.load('Wishfeatures',split='train')

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/wishfeatures/1.0.0.incompleteDI340D/wishfeatures-train.tfrecord*...:   0%|…

Dataset wishfeatures downloaded and prepared to /root/tensorflow_datasets/wishfeatures/1.0.0. Subsequent calls will reuse this data.


In [11]:
userEmbedding = userEmbeddingDs.map(lambda x:{
    'user_id' : x['user_id'],
    'user_domicile' : x['user_domicile'],
    'user_gender' : x['user_gender']
})

In [12]:
ratings = userFeatureDs.map(lambda x: {
    "location_name": x["location_name"],
    'location_id': x['location_id'],
    "user_id": x["user_id"],
    "add" : x["add"],
    "like" : x["like"]
})

In [13]:
locations2 = wishEmbeddingDs.map(lambda x: {
    'location_id' : x["location_id"],
    'location_name' : x['location_name']
    })

In [14]:
locations = wishEmbeddingDs.map(lambda x: x['location_name'])

In [15]:
locFeatures = wishFeaturesDs.map(lambda x:{
    'location_id' : x['location_id'],
    'like_count' : x['like_count'],
    'add_count' : x['add_count'],
    'rating' : x['rating']
})

In [16]:
for x in userEmbeddingDs.take(2).as_numpy_iterator():
  pprint.pprint(x)

{'user_age': 21,
 'user_domicile': b'Jawa Barat',
 'user_gender': b'Perempuan',
 'user_id': b'39',
 'user_occupation': b'Employees'}
{'user_age': 35,
 'user_domicile': b'DKI Jakarta',
 'user_gender': b'Laki-Laki',
 'user_id': b'135',
 'user_occupation': b'Employees'}


In [17]:
for x in userFeatureDs.take(2).as_numpy_iterator():
  pprint.pprint(x)

{'add': 0,
 'category': b'culinary',
 'like': 1,
 'location_id': b'61',
 'location_name': b'JAKARTA',
 'user_id': b'93'}
{'add': 0,
 'category': b'beach',
 'like': 1,
 'location_id': b'53',
 'location_name': b'DUBAI',
 'user_id': b'112'}


In [18]:
NUM_DATA = ratings.__len__().numpy()

tf.random.set_seed(42)
shuffled = ratings.shuffle(NUM_DATA, seed=42, reshuffle_each_iteration=False)

trainset_size = 0.8 * NUM_DATA

train = shuffled.take(trainset_size)
test = shuffled.skip(trainset_size).take(NUM_DATA - trainset_size)

location_name = locations.batch(1000)
user_ids = ratings.batch(1000).map(lambda x: x["user_id"])

unique_location_name = np.unique(np.concatenate(list(location_name)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

## MODEL

There are two critical parts to multi-task recommenders:

1. They optimize for two or more objectives, and so have two or more losses.
2. They share variables between the tasks, allowing for transfer learning.

In [32]:
class NewModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 32
    

    # User and movie models.
    self.location_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_location_name, mask_token=None),
      tf.keras.layers.Embedding(len(unique_location_name) + 1, embedding_dimension),
      tf.keras.layers.Dense(32, activation="relu")
    ])
    
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
      # tf.keras.layers.GRU(128, activation="relu")
    ])

    self.locFeatures_model : tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_location_name, mask_token=None
      ),
      # tf.keras.layers.Dense(16, activation='relu'),
      # tf.keras.layers.Dense(64, activation="relu"),
      # tf.keras.layers.Dense(1, activation = "sigmoid"),
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(1, activation = "sigmoid"),
    ])



    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=locations.batch(128).map(self.location_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model.
    location_embeddings = self.location_model(features["location_name"])
    
    return (
        user_embeddings,
        location_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings,location_embeddings], axis=1)
        ),
        self.locFeatures_model(
            # wishFeaturesDs
        )
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("add")

    user_embeddings, location_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, location_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

#### Hide

In [20]:
class RetrievalModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 32
    

    # User and movie models.
    self.location_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_location_name, mask_token=None),
      tf.keras.layers.Embedding(len(unique_location_name) + 1, embedding_dimension),
      tf.keras.layers.GRU(32)
    ])
    
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension),
      tf.keras.layers.Dense(32, activation="relu")
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(1, activation = "sigmoid"),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=locations.batch(128).map(self.location_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model.
    location_embeddings = self.location_model(features["location_name"])
    
    return (
        user_embeddings,
        location_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings,location_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("add")

    user_embeddings, location_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, location_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

### Rating-specialized model

Depending on the weights we assign, the model will encode a different balance of the tasks. Let's start with a model that only considers ratings.

In [27]:
model = NewModel(rating_weight=2.0, retrieval_weight=0.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
# model.build(input_shape=[])
# model.summary()

In [28]:
cached_train = train.shuffle(NUM_DATA).batch(512).cache()
cached_test = test.batch(256).cache()

In [29]:
model.fit(cached_train, epochs=10)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/10


ValueError: ignored

### Retrieval-specialized model

Let's now try a model that focuses on retrieval only.

#### Model V2

In [ ]:
model1 = RetrievalModel(rating_weight=0.0, retrieval_weight=1.0)
model1.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [ ]:
model1.fit(cached_train, epochs=10)
metrics = model1.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

In [ ]:
model1.evaluate(cached_test, return_dict=True)

#### Model V2.1

In [33]:
model2 = NewModel(rating_weight=0.0, retrieval_weight=1.0)
model2.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [ ]:
model2.fit(cached_train, epochs=10)
metrics2 = model2.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics2['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics2['root_mean_squared_error']:.3f}.")

In [ ]:
model2.evaluate(cached_test, return_dict=True)

### Joint model

Let's now train a model that assigns positive weights to both tasks.

In [ ]:
model = RetrievalModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [ ]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

# SEQUENTIAL

## model Class

In [ ]:
embedding_dimension = 32

# Query Model
user_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None
    ),
    tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension),
    tf.keras.layers.GRU(embedding_dimension)
])

# Candidate Model
wish_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary=unique_location_name, mask_token=None
    ),
    tf.keras.layers.Embedding(len(unique_location_name)+1, embedding_dimension)
]) 

In [ ]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=locations.batch(128).map(wish_model)
)

task = tfrs.tasks.Retrieval(
  metrics=metrics
)

class Model_V2_1(tfrs.Model):

    def __init__(self, query_model, candidate_model):
        super().__init__()
        self._query_model = query_model
        self._candidate_model = candidate_model

        self._task = task
    
    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
      # We pick out the user features and pass them into the user model.
      user_embeddings = self.user_model(features["user_id"])
      # And pick out the movie features and pass them into the movie model,
      # getting embeddings back.
      positive_location_embeddings = self.location_model(features["location_name"])

      # The task computes the loss and the metrics.
      return self.task(user_embeddings, positive_location_embeddings)

In [ ]:
model = Model_V2_1(user_model, wish_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
cached_train = train.shuffle(NUM_DATA).batch(512).cache()
cached_test = test.batch(256).cache()

In [ ]:
model.fit(cached_train, epochs=3)

Epoch 1/3


NotImplementedError: ignored